In [24]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
from us import states

In [85]:
def rename_location(fire):
    name = fire.partition(" COUNTY")
    fire = name[0] + name[1]
    return fire
def filter_counties(list1, list2):
    return [n for n in list1 if
             any(m in n for m in list2)]

In [92]:
# Importing csv files for wildfires and for census years.
wildfires = pd.read_csv("NOAA_CSVs/wildfires_2010_2019.csv")
census_data_2010 = pd.read_csv(f"Census_csvs/2010_census_data_ca_county.csv")
census_data_2011 = pd.read_csv(f"Census_csvs/2011_census_data_ca_county.csv")
census_data_2012 = pd.read_csv(f"Census_csvs/2012_census_data_ca_county.csv")
census_data_2013 = pd.read_csv(f"Census_csvs/2013_census_data_ca_county.csv")
census_data_2014 = pd.read_csv(f"Census_csvs/2014_census_data_ca_county.csv")
census_data_2015 = pd.read_csv(f"Census_csvs/2015_census_data_ca_county.csv")
census_data_2016 = pd.read_csv(f"Census_csvs/2016_census_data_ca_county.csv")
census_data_2017 = pd.read_csv(f"Census_csvs/2017_census_data_ca_county.csv")
census_data_2018 = pd.read_csv(f"Census_csvs/2018_census_data_ca_county.csv")
census_data_2019 = pd.read_csv(f"Census_csvs/2019_census_data_ca_county.csv")

0      05/20/2010
1      07/13/2010
2      07/15/2010
3      07/15/2010
4      07/15/2010
          ...    
393    11/01/2019
394    11/01/2019
395    11/01/2019
396    11/01/2019
397    11/01/2019
Name: BEGIN_DATE, Length: 398, dtype: object

In [88]:
# PROBABLY WANT TO USE THIS ELSEWHERE LATER ON BUT FOR NOW WANT ALL COLUMNS ------------------------------
# Makes smaller DataFrame from wildfires for relevant columns.
# wildfires = wildfires[['EVENT_ID','CZ_NAME_STR','CZ_FIPS']]

# Variables for parsing lists.
fire = ""
new_names = []
chosen_names = []
substring = "(ZONE)"
census_names = [census_data_2010["Name"]]

# Parses through both lists of counties and pulls out only shared values.
for fire in wildfires["CZ_NAME_STR"]:
    fire = rename_location(fire)
    new_names.append(fire)
for name in new_names:
    if substring not in name:
        if name not in chosen_names:
            chosen_names.append(name.title())
for name in census_data_2010['Name']:
    census_names.append(name.title())
list1 = census_names
list2 = chosen_names
utile_counties = filter_counties(list1, list2)

In [89]:
# Reduces NOAA data to rows found in the usable counties.
wildfires

,EVENT_ID,CZ_NAME_STR,CZ_FIPS
0,234746,SAN BERNARDINO AND RIVERSIDE COUNTY VALLEYS - ...,48
1,250167,SAN DIEGO COUNTY COASTAL AREAS (ZONE),43
2,250175,SAN BERNARDINO AND RIVERSIDE COUNTY VALLEYS - ...,48
3,250169,RIVERSIDE COUNTY MOUNTAINS (ZONE),56
4,250171,SAN BERNARDINO AND RIVERSIDE COUNTY VALLEYS - ...,48
...,...,...,...
393,863910,VENTURA COUNTY COASTAL VALLEYS (ZONE),45
394,863913,VENTURA COUNTY INTERIOR VALLEYS (ZONE),44
395,863914,VENTURA COUNTY COASTAL VALLEYS (ZONE),45
396,863915,VENTURA COUNTY COAST (ZONE),40
